In [149]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import Image

In [150]:
def obtener_info(s_notas):
    diccionario = {}
    #Extraemos la fecha
    fecha = s_notas.find('span',attrs={'pubdate':'pubdate'})
    if fecha:
        diccionario['Fecha'] = fecha.get('datetime')
    else:
        diccionario['Fecha'] = None
     #Extraemos el titulo
    titulo = s_notas.find('h1',attrs={'class':'article-title'})
    if titulo:
        diccionario['Titulo'] = titulo.text
    else:
        diccionario['Titulo'] = None
    #Extraemos la volanda
    volanda = s_notas.find('h2',attrs={'class':'article-prefix'})
    if volanda:
        diccionario['Volanda'] = volanda.text
    else:
        diccionario['Volanda'] = None
    #Extraemos el copete
    copete = s_notas.find('div',attrs={'class':'article-summary'})
    if copete:
        diccionario['Copete'] = copete.text
    else:
        diccionario['Copete'] = None
    #Extraemos el autor
    autor = s_notas.find('div',attrs={'class':'article-author'})
    if autor:
        diccionario['Autor'] = autor.text
    else:
        diccionario['Autor'] = None
    #Extraemos la imagen
    media = s_notas.find('div',attrs={'class':'article-main-media'})
    if media:
        imagenes = media.find_all('img')
        if len(imagenes)==0:
            print('no se encontro imagen')
        else:
            imagen = imagenes[-1]
            img_src = imagen.get('data-src')
            try:
                img_get = requests.get(img_src)
                if img_get.status_code == 200:
                    diccionario['Imagen'] = img_get.content
                else:
                    diccionario['Imagen'] = None
            except:
                print('No se encontro imagen')
    #Extremos el cuerpo:
    cuerpo  = s_notas.find('div',attrs={'class':'article-text'})
    if cuerpo:
        diccionario['Cuerpo de texto'] = cuerpo.text
    else:
        diccionario['Cuerpo de texto'] = None
    return diccionario

In [151]:
def scrapy_nota(url):
    try:
        req = requests.get(url)
    except Exception as e:
        print(f'error: {e}')
        print(f'url: {url}')
        return None
    if req.status_code != 200:
        print(f'eror en status, hubo un error de {req.status_code}')
        return None
    s_nota = BeautifulSoup(req.text,'lxml')
    diccionario = obtener_info(s_nota)
    diccionario['Url'] = url
    return diccionario

In [152]:
url_nota = 'https://www.pagina12.com.ar/279366-santiago-cafiero-el-presidente-va-a-seguir-dialogando-con-to'

In [153]:
url_nota

'https://www.pagina12.com.ar/279366-santiago-cafiero-el-presidente-va-a-seguir-dialogando-con-to'

In [154]:
scrapy_nota(url_nota)

{'Fecha': '2020-07-18',
 'Titulo': 'Santiago Cafiero: "El Presidente va a seguir dialogando" con todos los sectores',
 'Volanda': 'Definiciones del jefe de Gabinete tras la carta de Hebe de Bonafini a Alberto Fernández',
 'Copete': 'También se refirió a la apertura escalonada en AMBA. "Si alguna de las variables que se analizan prenden luces de alarma, vamos a volver para atrás", afirmó, y apeló a la "responsabilidad individual" para la nueva etapa.',
 'Autor': None,
 'Imagen': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc2\x00\x11\x08\x02\x80\x03\xc0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1b\x00\x00\x02\x03\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [155]:
def links(soup):
    list_links= list()
    cont = soup.find('div',attrs={'class':'featured-article__container'})
    link_container = cont.a.get('href')
    web_ul = soup.find('ul',attrs={'class':'article-list'})
    list_links.append(link_container)
    for link in web_ul.find_all('h2'):
        list_links.append(link.a.get('href'))
    return list_links

In [156]:
url =  'https://www.pagina12.com.ar/secciones/el-pais'
ps2 = requests.get(url)
soup = BeautifulSoup(ps2.text, 'lxml')
links_list = links(soup)

In [157]:
links_list

['https://www.pagina12.com.ar/279202-quienes-pueden-usar-el-transporte-publico-en-amba-en-la-nuev',
 'https://www.pagina12.com.ar/274834-cuales-son-las-30-actividades-esenciales-hasta-el-2-de-agost',
 'https://www.pagina12.com.ar/279366-santiago-cafiero-el-presidente-va-a-seguir-dialogando-con-to',
 'https://www.pagina12.com.ar/279353-alberto-fernandez-recordo-el-atentado-a-la-amia-en-su-aniver',
 'https://www.pagina12.com.ar/279351-rodriguez-larreta-hay-que-aprender-a-convivir-con-el-virus-c',
 'https://www.pagina12.com.ar/279234-el-represor-argentino-que-podra-ser-juzgado-en-italia',
 'https://www.pagina12.com.ar/279240-localizan-al-represor-argentino-que-se-ocultaba-en-berlin',
 'https://www.pagina12.com.ar/279272-los-gobernadores-acompanaron-al-presidente-y-pidieron-respon',
 'https://www.pagina12.com.ar/279278-el-cruce-de-mario-cimadevilla-con-mauricio-macri-por-la-amia',
 'https://www.pagina12.com.ar/279284-atentado-a-la-amia-el-acto-oficial-la-acusacion-contra-iran-',
 'https://

In [158]:
data = []
for i, nota in enumerate(links_list):
    print(f'Scrapeando nota{i}/{len(links_list)}')
    data.append(scrapy_nota(nota))

Scrapeando nota0/15
Scrapeando nota1/15
Scrapeando nota2/15
Scrapeando nota3/15
Scrapeando nota4/15
Scrapeando nota5/15
Scrapeando nota6/15
Scrapeando nota7/15
Scrapeando nota8/15
Scrapeando nota9/15
Scrapeando nota10/15
Scrapeando nota11/15
Scrapeando nota12/15
Scrapeando nota13/15
Scrapeando nota14/15


In [162]:
df = pd.DataFrame(data)

In [163]:
df

,Fecha,Titulo,Volanda,Copete,Autor,Imagen,Cuerpo de texto,Url
0,2020-07-18,Quiénes pueden usar el transporte público en A...,Los trabajadores esenciales de acuerdo al nuev...,None,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,A pesar de la flexibilización de las restricci...,https://www.pagina12.com.ar/279202-quienes-pue...
1,2020-07-18,Cuáles son las 30 actividades esenciales hasta...,Los trabajadores y trabajadoras esenciales par...,None,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Con el objetivo de controlar los casos de covi...,https://www.pagina12.com.ar/274834-cuales-son-...
2,2020-07-18,"Santiago Cafiero: ""El Presidente va a seguir d...",Definiciones del jefe de Gabinete tras la cart...,También se refirió a la apertura escalonada en...,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"El jefe de Gabinete, Santiago Cafiero, subrayó...",https://www.pagina12.com.ar/279366-santiago-ca...
3,2020-07-18,Alberto Fernández recordó el atentado a la AMI...,"""La búsqueda de verdad y justicia es un impera...","""Los argentinos tenemos una deuda que debemos ...",None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Al cumplirse hoy 26 años del ataque terrorista...,https://www.pagina12.com.ar/279353-alberto-fer...
4,2020-07-18,"Rodríguez Larreta: ""Hay que aprender a convivi...",Definiciones del jefe de gobierno porteño sobr...,"Instó a no reunirse el Día del Amigo, que se c...",None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,"El jefe de Gobierno porteño, Horacio Rodríguez...",https://www.pagina12.com.ar/279351-rodriguez-l...
5,2020-07-18,El represor argentino que podrá ser juzgado en...,El prófugo Carlos Malatto había huido a Italia...,None,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Si no fuera por la pandemia del covid-19 el re...,https://www.pagina12.com.ar/279234-el-represor...
6,2020-07-18,Localizan al represor argentino que se ocultab...,El exmarino Luis Esteban Kyburg había huido ha...,Tiene ciudadanía alemana y por eso no se otorg...,Por Adriana Meyer,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Sería una paradoja intolerable para Alemania s...,https://www.pagina12.com.ar/279240-localizan-a...
7,2020-07-18,Los gobernadores acompañaron al Presidente y p...,"La continuidad del aislamiento en Río Negro, J...",None,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,El anuncio de Alberto Fernández sobre la flexi...,https://www.pagina12.com.ar/279272-los-goberna...
8,2020-07-18,El cruce de Mario Cimadevilla con Mauricio Mac...,"""Memoria, Verdad y Justicia nunca estuvieron e...",None,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,Mario Cimedevilla fue titular de la Unidad de ...,https://www.pagina12.com.ar/279278-el-cruce-de...
9,2020-07-18,"Atentado a la AMIA: el acto oficial, la acusac...",A 26 años del ataque a la mutual judía,En el homenaje organizado por la AMIA y la DAI...,Por Raúl Kollmann,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,El acto oficial a 26 años del atentado a la AM...,https://www.pagina12.com.ar/279284-atentado-a-...


In [164]:
df.to_csv('Pagina12.csv')